In [1]:
# SELF REMINDER. Copy the 'ocpa' directory to the forked one from github, so that I can push updates to github.
# Python native
import ast
import pickle
from statistics import median, mean
import pandas as pd
import numpy as np
from copy import copy
# Data handling
# Object centric process mining
# import ocpa.algo.evaluation.precision_and_fitness.utils as evaluation_utils # COMMENTED OUT BY TIM
# import ocpa.algo.evaluation.precision_and_fitness.evaluator as precision_fitness_evaluator # COMMENTED OUT BY TIM
import ocpa.objects.log.importer.csv.factory as csv_import_factory
import ocpa.algo.predictive_monitoring.factory as feature_factory
from ocpa.objects.log.ocel import OCEL
from ocpa.algo.predictive_monitoring.obj import Feature_Storage as FeatureStorage

# # Simple machine learning models, procedure tools, and evaluation metrics
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# # Custom GNN tools
# from gnn_utils import (
#     generate_graph_dataset,
#     # get_ordered_event_list,
#     # visualize_graph,
#     # show_remaining_times,
#     # visualize_instance,
#     # GraphDataLoader,
#     # GCN,
#     # evaluate_gnn,
# )

# PyG
import torch
from torch_geometric.loader import DataLoader
# from replicating.ocpa_PyG_integration.EventGraphDataset import EventGraphDataset
from replicating.ocpa_PyG_integration.EventSubGraphDataset import EventSubGraphDataset
from replicating.model import GCN, GAT

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

# Global variables
from replicating.experiment_config import STORAGE_PATH, FEATURE_STORAGE_FILE, RANDOM_SEED, TARGET_LABEL


filename = "data/adams/example_logs/mdl/BPI2017-Final.csv"
object_types = ["application", "offer"]
parameters = {
    "obj_names": object_types,
    "val_names": [],
    "act_name": "event_activity",
    "time_name": "event_timestamp",
    "sep": ",",
}
file_path_object_attribute_table = None

c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.13.1+cpu
Cuda available: False
Torch geometric version: 2.2.0


In [2]:
with open(
        f"{STORAGE_PATH}/raw/BPI17-feature_storage-[C2,D1,P2,P3,O3].fs", "rb"
    ) as file:
        fs: FeatureStorage = pickle.load(file)

# Adams didn't give this split a random seed, 
# thus we can split the validation set in this arbitrary manner
fs.extract_normalized_train_test_split(
    test_size=0.3,
    validation_size=0.7*0.2, 
    scaler=StandardScaler,
    scaling_exempt_features=[],
    state=RANDOM_SEED,
)

c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\pandas\core\common.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged

In [3]:
test_mae_normed = 0.4386
prepared_normalized_data = {TARGET_LABEL:[test_mae_normed, 0.87 ,0.222,2.7]}
normalized_data = {'1prev_act':[-1,-2],TARGET_LABEL:[test_mae_normed, test_mae_normed+1],'horse':[1,1],'WRONG':[5,5]}
event_features = ['prev_act',TARGET_LABEL,'paart','horse','rem_time']
fs.scaler.inverse_transform([test_mae_normed]*25)[-2]

1706496.518711365

In [5]:
# KLOPT NOG NIET
res = fs.denormalize(normalized_data)
pd.DataFrame(res)

c:\Users\Tim\Development\OCELFeatureExtractionExperiments\.env\lib\site-packages\ocpa\algo\predictive_monitoring\obj.py:316: UserWarning: Could not find keys '{'horse', 'WRONG', '1prev_act'}' as event features in Feature_Storage. They will be excluded from denormalization.
  warn(


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.079982,0.051701,0.076074,0.079982,0.216048,0.100262,0.138106,0.100674,0.059312,0.059181,...,0.00936,0.032419,0.005267,0.001011,0.000014,0.000181,16479.460734,732939.172227,1.181379e+06,1.432241
1,0.079982,0.051701,0.076074,0.079982,0.216048,0.100262,0.138106,0.100674,0.059312,0.059181,...,0.00936,0.032419,0.005267,0.001011,0.000014,0.000181,16479.460734,732939.172227,1.181379e+06,2.319387


In [12]:
empty_dict = copy(fs.empty_dict)
len(list(empty_dict.values())[0])

4

In [49]:

# prepared_normalized_data
# empty_dict.pop(TARGET_LABEL)
empty_dict.update(prepared_normalized_data)
h = pd.DataFrame(empty_dict)
res = fs.scaler.inverse_transform(h)

{1: [0, 0, 0, 0],
 2: [0, 0, 0, 0],
 3: [0, 0, 0, 0],
 4: [0, 0, 0, 0],
 5: [0, 0, 0, 0],
 6: [0, 0, 0, 0],
 7: [0, 0, 0, 0],
 8: [0, 0, 0, 0],
 9: [0, 0, 0, 0],
 10: [0, 0, 0, 0],
 11: [0, 0, 0, 0],
 12: [0, 0, 0, 0],
 13: [0, 0, 0, 0],
 14: [0, 0, 0, 0],
 15: [0, 0, 0, 0],
 16: [0, 0, 0, 0],
 17: [0, 0, 0, 0],
 18: [0, 0, 0, 0],
 19: [0, 0, 0, 0],
 20: [0, 0, 0, 0],
 21: [0, 0, 0, 0],
 22: [0, 0, 0, 0],
 24: [0, 0, 0, 0],
 ('event_remaining_time', ()): [0.4386, 1.4386],
 0: [0, 0]}

In [11]:
normalized_data_keys = [(i, str(key)) for i,key in enumerate(normalized_data.keys()) if type(key)==tuple]
# original_key_idxs = np.argsort(normalized_data_keys)

# [ast.literal_eval(key) for key in normalized_data_keys]
normalized_data_keys

[(1, "('event_remaining_time', ())")]

In [12]:
invalid_keys = set(normalized_data.keys()) - set(event_features)
valid_keys = set(normalized_data.keys()) & set(event_features)
valid_keys

{('event_remaining_time', ()), 'horse', 'prev_act'}

In [48]:

df = pd.DataFrame(normalized_data)
valid_key_idxs_in_event_feats = {event_features.index(key) for key in valid_keys}
event_feats_idxs = set(range(len(event_features)))
absent_feat_idxs_in_normed_data_keys = event_feats_idxs - valid_key_idxs_in_event_feats
empty_dict = {k:v for (k,v) in zip(range(len(event_features)),[[0]*len(df)]*len(event_features))}

for key_idx,key in zip(valid_key_idxs_in_event_feats,valid_keys):
    for k,v in empty_dict.items():
        if key_idx ==k:
            empty_dict.update({key_idx:normalized_data[key]})
    empty_dict[key] = empty_dict.pop(key_idx)

df = pd.DataFrame(empty_dict)
set(df.keys())

{('event_remaining_time', ()), 2, 4, 'horse', 'prev_act'}

In [35]:
normalized_data[TARGET_LABEL]

[0.4386, 1.4386]

In [8]:
import ast
tup_to_str = str(TARGET_LABEL)
ast.literal_eval(tup_to_str)

('event_remaining_time', ())

In [9]:
fs.ugly_boy.head()

,event_id,"(event_preceding_activities, (Create application,))","(event_preceding_activities, (Submit,))","(event_preceding_activities, (Complete,))","(event_preceding_activities, (Accept,))","(event_preceding_activities, (Create offer,))","(event_preceding_activities, (Send (mail and online),))","(event_preceding_activities, (Call,))","(event_preceding_activities, (Validate,))","(event_preceding_activities, (Return,))",...,"(event_preceding_activities, (Handle leads,))","(event_preceding_activities, (Cancel application,))","(event_preceding_activities, (Send (online),))","(event_preceding_activities, (Assess potential fraud,))","(event_preceding_activities, (Personal loan collection,))","(event_preceding_activities, (Shorten completion,))","(event_aggregate_previous_char, (event_RequestedAmount, <built-in function max>))","(event_elapsed_time, ())","(event_remaining_time, ())","(event_previous_type_count, (offer,))"
0,236558,-0.294847,-0.233496,-0.286946,-0.294847,-0.523023,-0.328672,-0.400249,-0.33458,-0.250309,...,-0.097205,-0.183044,-0.070832,-0.031809,-0.003688,-0.013466,-1.069468,-0.710438,-0.266181,-1.175837
1,236559,3.391587,-0.233496,-0.286946,-0.294847,-0.523023,-0.328672,-0.400249,-0.33458,-0.250309,...,-0.097205,-0.183044,-0.070832,-0.031809,-0.003688,-0.013466,-1.069468,-0.710438,-0.266181,-1.175837
2,236560,-0.294847,-0.233496,3.484972,-0.294847,-0.523023,-0.328672,-0.400249,-0.33458,-0.250309,...,-0.097205,-0.183044,-0.070832,-0.031809,-0.003688,-0.013466,-1.069468,-0.710314,-0.266287,-1.175837
3,236561,-0.294847,-0.233496,-0.286946,3.391587,-0.523023,-0.328672,-0.400249,-0.33458,-0.250309,...,-0.097205,-0.183044,-0.070832,-0.031809,-0.003688,-0.013466,-1.069468,-0.626416,-0.338582,-0.048627
4,236562,-0.294847,-0.233496,-0.286946,-0.294847,1.897842,-0.328672,-0.400249,-0.33458,-0.250309,...,-0.097205,-0.183044,-0.070832,-0.031809,-0.003688,-0.013466,-1.069468,-0.626407,-0.338590,-0.048627
